## 移除最后conv最后两层网络


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 256
num_epochs = 300
extract_size = 512
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut2Layer()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_2_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_2_conv.h')

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [4]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = FinalClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### -> Load the best trained model,yet!

In [5]:
classify_model.load_state_dict(torch.load('./models/class_single_rm2layer_last.pkl'))

### time to train model

In [6]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [7]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(301, num_epochs+301):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm2layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm2layer_last.pkl'))

Epoch [301/300], Step [120/127], Loss: 1.7130, Accuracy: 61.72%, Best_acc: 61.72%            
 Epoch 301, spent time:5.58s
Epoch [302/300], Step [120/127], Loss: 1.8141, Accuracy: 55.86%, Best_acc: 61.72%            
 Epoch 302, spent time:5.53s
Epoch [303/300], Step [120/127], Loss: 1.8334, Accuracy: 58.59%, Best_acc: 61.72%            
 Epoch 303, spent time:5.68s
Epoch [304/300], Step [120/127], Loss: 1.7482, Accuracy: 58.98%, Best_acc: 61.72%            
 Epoch 304, spent time:5.59s
Epoch [305/300], Step [120/127], Loss: 1.8079, Accuracy: 58.98%, Best_acc: 61.72%            
 Epoch 305, spent time:5.65s
Epoch [306/300], Step [120/127], Loss: 1.8819, Accuracy: 55.08%, Best_acc: 61.72%            
 Epoch 306, spent time:5.63s
Epoch [307/300], Step [120/127], Loss: 1.8332, Accuracy: 56.25%, Best_acc: 61.72%            
 Epoch 307, spent time:5.56s
Epoch [308/300], Step [120/127], Loss: 1.7614, Accuracy: 61.33%, Best_acc: 63.28%            
 Epoch 308, spent time:5.62s
Epoch [309/300],

Epoch [367/300], Step [120/127], Loss: 1.7514, Accuracy: 61.72%, Best_acc: 64.84%            
 Epoch 367, spent time:5.68s
Epoch [368/300], Step [120/127], Loss: 1.8602, Accuracy: 53.91%, Best_acc: 64.84%            
 Epoch 368, spent time:5.57s
Epoch [369/300], Step [120/127], Loss: 1.7566, Accuracy: 58.20%, Best_acc: 64.84%            
 Epoch 369, spent time:5.54s
Epoch [370/300], Step [120/127], Loss: 1.8022, Accuracy: 59.38%, Best_acc: 64.84%            
 Epoch 370, spent time:5.64s
Epoch [371/300], Step [120/127], Loss: 1.7701, Accuracy: 60.16%, Best_acc: 64.84%            
 Epoch 371, spent time:5.57s
Epoch [372/300], Step [120/127], Loss: 1.8039, Accuracy: 55.47%, Best_acc: 64.84%            
 Epoch 372, spent time:5.62s
Epoch [373/300], Step [120/127], Loss: 1.9022, Accuracy: 49.61%, Best_acc: 64.84%            
 Epoch 373, spent time:5.56s
Epoch [374/300], Step [120/127], Loss: 1.8009, Accuracy: 56.64%, Best_acc: 64.84%            
 Epoch 374, spent time:5.63s
Epoch [375/300],

Epoch [433/300], Step [120/127], Loss: 1.8075, Accuracy: 60.55%, Best_acc: 67.97%            
 Epoch 433, spent time:5.66s
Epoch [434/300], Step [120/127], Loss: 1.7754, Accuracy: 61.72%, Best_acc: 67.97%            
 Epoch 434, spent time:5.60s
Epoch [435/300], Step [120/127], Loss: 1.6647, Accuracy: 62.50%, Best_acc: 67.97%            
 Epoch 435, spent time:5.62s
Epoch [436/300], Step [120/127], Loss: 1.7607, Accuracy: 57.81%, Best_acc: 67.97%            
 Epoch 436, spent time:5.67s
Epoch [437/300], Step [120/127], Loss: 1.9607, Accuracy: 52.34%, Best_acc: 67.97%            
 Epoch 437, spent time:5.57s
Epoch [438/300], Step [120/127], Loss: 1.7846, Accuracy: 56.64%, Best_acc: 67.97%            
 Epoch 438, spent time:5.58s
Epoch [439/300], Step [120/127], Loss: 1.7847, Accuracy: 59.38%, Best_acc: 67.97%            
 Epoch 439, spent time:5.65s
Epoch [440/300], Step [120/127], Loss: 1.7900, Accuracy: 57.81%, Best_acc: 67.97%            
 Epoch 440, spent time:5.55s
Epoch [441/300],

Epoch [499/300], Step [120/127], Loss: 1.6955, Accuracy: 60.55%, Best_acc: 67.97%            
 Epoch 499, spent time:5.55s
Epoch [500/300], Step [120/127], Loss: 1.8726, Accuracy: 57.42%, Best_acc: 67.97%            
 Epoch 500, spent time:5.59s
Epoch [501/300], Step [120/127], Loss: 1.8603, Accuracy: 56.25%, Best_acc: 67.97%            
 Epoch 501, spent time:5.60s
Epoch [502/300], Step [120/127], Loss: 1.7673, Accuracy: 60.55%, Best_acc: 67.97%            
 Epoch 502, spent time:5.57s
Epoch [503/300], Step [120/127], Loss: 1.7477, Accuracy: 58.98%, Best_acc: 67.97%            
 Epoch 503, spent time:5.71s
Epoch [504/300], Step [120/127], Loss: 1.7519, Accuracy: 59.77%, Best_acc: 67.97%            
 Epoch 504, spent time:5.64s
Epoch [505/300], Step [120/127], Loss: 1.7604, Accuracy: 62.11%, Best_acc: 67.97%            
 Epoch 505, spent time:5.64s
Epoch [506/300], Step [120/127], Loss: 1.8064, Accuracy: 55.47%, Best_acc: 67.97%            
 Epoch 506, spent time:5.63s
Epoch [507/300],

Epoch [565/300], Step [120/127], Loss: 1.8420, Accuracy: 54.30%, Best_acc: 67.97%            
 Epoch 565, spent time:5.63s
Epoch [566/300], Step [120/127], Loss: 1.8213, Accuracy: 59.77%, Best_acc: 67.97%            
 Epoch 566, spent time:5.70s
Epoch [567/300], Step [120/127], Loss: 1.7452, Accuracy: 60.55%, Best_acc: 67.97%            
 Epoch 567, spent time:5.58s
Epoch [568/300], Step [120/127], Loss: 1.8036, Accuracy: 55.47%, Best_acc: 67.97%            
 Epoch 568, spent time:5.62s
Epoch [569/300], Step [120/127], Loss: 1.7362, Accuracy: 58.98%, Best_acc: 67.97%            
 Epoch 569, spent time:5.63s
Epoch [570/300], Step [120/127], Loss: 1.6830, Accuracy: 62.50%, Best_acc: 67.97%            
 Epoch 570, spent time:5.59s
Epoch [571/300], Step [120/127], Loss: 1.7612, Accuracy: 62.11%, Best_acc: 67.97%            
 Epoch 571, spent time:5.58s
Epoch [572/300], Step [120/127], Loss: 1.7579, Accuracy: 60.16%, Best_acc: 67.97%            
 Epoch 572, spent time:5.53s
Epoch [573/300],

### How good is the model?

In [8]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm2layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm2layer.h")

In [29]:
classify_model.load_state_dict(torch.load('./models/class_single_rm2layer_last.pkl'))

In [9]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 54.26%.
